In [1]:
# syft absolute
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=cc7343ca948646e6b8374e3a193d1968 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=3cb80d4c8c2c499989fdcf8644d5a002 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

# create datasets

In [4]:
# third party
import numpy as np

mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=private_low.shape,
            mock_is_real=True,
        )
    ],
)

client_low.upload_dataset(dataset_low)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.46it/s]


SyftSuccess: Dataset uploaded to 'test_l'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [5]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    id=dataset_low.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.01it/s]


Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [6]:
data_low = client_low.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/525cfcf0a67b4a71b7056902d6f7937a.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=bc19b4049e464513b1597ad3601a9304 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_717: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_717
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [7]:
client_low.code.request_code_execution(compute_mean)

SyftInfo: Node Landed!


syft.service.request.request.Request

In [8]:
request = client_low.requests[0]

In [9]:
code = client_low.code[0]

TEO HERE

In [10]:
items_to_sync = [request, code]

- What do we do with properties here?
  - just set them to None

In [11]:
def sync_items(items):
    for item in items_to_sync:
        services = list(node_high.python_node.service_path_map.values())

        all_stashes = dict()
        for serv in services:
            if (_stash := getattr(serv, "stash", None)) is not None:
                all_stashes[_stash.object_type] = _stash

        stash = all_stashes.get(type(item), None)

        res = stash.set(node_high.python_node.verify_key, item)
        if not res.is_ok():
            raise ValueError("")
        else:
            print("created", item)

In [13]:
sync_items(items_to_sync)

created syft.service.request.request.Request
created syft.service.code.user_code.UserCode


In [13]:
res

Err("<class 'syft.service.code.user_code.UserCode'> does not match required type: <class 'syft.service.request.request.Request'>")

In [13]:
res

Err('Duplication Key Error for syft.service.request.request.Request.\nThe fields that should be unique are `request_hash`, `id`.')

In [28]:
list(node_high.python_node.document_store.partitions.keys())[0]

'QueueItem'

In [ ]:
client_high.